In [3]:
import numpy as np
import pandas as pd
import scipy.io

from ruspy.estimation.estimation import estimate
from promotion.replication.auxiliary_iskhakov_et_al_2016.auxiliary import (process_data, process_result)

---
# Replication of Iskhakov et al. (2016)
---

In this notebook we seek to replicate Iskhakov et al. (2016) with some notable differences in the setup which we will explain in the course of this notebook. To familiarize you with the set up in the paper let us start with a few key points to understand the remainder. Iskhakov et al. (2016) follow up on Su and Judd (2012) in which they had compared the performance of Mathematical Programming with Equlibrium Constraints (MPEC) and the Nested Fixed Point Algorithm (NFXP) for the canonical structural model of bus engine replacement based on Rust (1987). The aim of Iskhakov et al. is mainly to improve the setup of the NFXP of Su and Judd following the explanations of Rust (2000). Their revised comparison between MPEC and NFXP shows that both approaches perform similarly for a Monte Carlo simulation of the bus engine replacement problem. 

We replicate this Monte Carlo simulation below using our ruspy package in which we have implemented MPEC as well as the NFXP in pure Python. This is gives us the flexibility to easily experiment with key assumptions such as tolerance or optimization algorithms used in one common interface. 

In the boxes just below we obtain all the key results needed to replicate Table I of Iskhakov et al. For that we got the simulated data by using the original matlab code of the paper to perform the simulation. There they use the setup in which we have 120 time periods and 50 buses. The cost function is assumed to be linear taking the following form $c(x, \theta_1) = 0.001 \theta_{11} x$. The true parameters are the following: 

\begin{equation}
RC=11.7257,  \\
\theta_{11}=2.4569, \\
\theta_3=(0.0937, 0.4475, 0.4459, 0.0127, 0.0002).
\end{equation}

The true parameter for $\beta$ is varied: $\beta\in \{0.975, 0.985, 0.995, 0.999, 0.9995, 0.9999\}$. For each $\beta$ 250 data sets are simulated. Those are then estimated using five different starting values for the replacement cost $RC$ and the linear cost parameter $\theta_{11}$. The following are the five different starting values: 

\begin{align}
(RC^0, \theta^0_{11}) \in \{(4,1), (5,2), (6,3), (7,4), (8,5)\}
\end{align}

In the case of MPEC also starting values for the discretized expected value function are necessary and they are set to the zero vector every time: $EV^0_1,...,EV^0_{175}=(0,...,0)$. The subscript of 175 also displays the grid size that Iskhakov et al. choose to discretize the continuous variable of mileage. The starting values of the transition probabilities are frequency based in the paper and we follow this approach in our implementation.

All those ingredients can be found below in the code. Further in our code for the NFXP we specify stopping criteria for the fixed point calculation as well as the tolerance at which we switch from contraction steps to Newton-Kantorovich (N-K) steps. Here, there is a first difference in our implementation of the NFXP as there is no switching back from N-K to contractions teps implemented. Furthermore the switching tolerance is solely an absolute one. For the routine to maximize the likelihood function we rely on the BHHH implementation of estimagic which uses a combination of relative and absolute stopping tolerance which also does not exactly match the original paper. 

For the MPEC we could not rely on KNITRO as in the paper as it is not freely available. In ruspy it is possible to choose from the library of nlopt or use ipopt for MPEC (please see the References below for links to the respective documentation). For the recreation of the table in Iskhakov et al. we decided to use ipopt here. Again the stopping tolerances cannot exactly be set to those of KNITRO in the paper. Another notable difference is that we only give analytical first order derivatives to ipopt. In the paper, on top of that second order analytical derivatives are provided by using automatic differentiation and also the sparsitiy patterns of both order derivatives are passed in. Apart from that, though, our setup replicates the paper by using upper and lower bounds as well as a properly recentering the expected value function. 

A last difference in our setup that affects both NFXP and MPEC is that we estimate the transition probabilities separately from the cost parameters. In the original paper for MPEC those are estimated jointly and for the NFXP the cost parameters are first estimated partially and then after that together with the transition probabilities.

For ruspy to run the estimation using either NFXP or MPEC we only need to pass a dictionairy plus the data to the function `estimate(init_dict, data)`. Below we create one dictionairy per approach (NFXP or MPEC) and adapt it accordingly within the nested for-loop below depending on which discount factor $\beta$ and which starting values are used in the respective run.

In [5]:
# Initialize the simulation
discount_factor = [0.975, 0.985, 0.995, 0.999, 0.9995, 0.9999]
approach = ["NFXP", "MPEC"]
starting_cost_params = np.vstack((np.arange(4,9), np.arange(1,6)))
starting_expected_value_fun = np.zeros(175)
number_runs = 250
number_buses = 50
number_periods = 120
number_states = 175
number_cost_params = 2

# Initialize the set up for the nested fixed point algorithm
stopping_crit_fixed_point = 1e-13
switch_tolerance_fixed_point = 1e-2

# Initialize the set up for MPEC
lower_bound = np.concatenate((np.full(number_states, -np.inf), np.full(number_cost_params, 0.0)))
upper_bound = np.concatenate((np.full(number_states, 500.0), np.full(number_cost_params, np.inf)))
rel_ipopt_stopping_tolerance = 1e-6

init_dict_nfxp = {
    "model_specifications": {
        "number_states": number_states,
        "maint_cost_func": "linear",
        "cost_scale": 1e-3,
    },
    "optimizer": {
        "approach": "NFXP",
        "algorithm": "estimagic_bhhh",
        # implies that we use analytical first order derivatives as opposed to numerical ones
        "gradient": "Yes",   
    },
    "alg_details": {
        "threshold": stopping_crit_fixed_point,
        "switch_tol": switch_tolerance_fixed_point,
    },
}

init_dict_mpec = {
    "model_specifications": {
        "number_states": number_states,
        "maint_cost_func": "linear",
        "cost_scale": 1e-3,
    },
    "optimizer": {
        "approach": "MPEC",
        "algorithm": "ipopt",
        # implies that we use analytical first order derivatives as opposed to numerical ones        
        "gradient": "Yes",   
        "tol": rel_ipopt_stopping_tolerance,
        "set_lower_bounds": lower_bound,
        "set_upper_bounds": upper_bound,
    },
}

In [3]:
# Initialize DataFrame to store the results of each run of the Monte Carlo simulation
index = pd.MultiIndex.from_product([discount_factor, 
                                    range(number_runs),
                                    range(starting_cost_params.shape[1]),
                                    approach],
                                   names=["Discount Factor", "Run", "Start", "Approach"])

columns=["RC", "theta_11", "theta_30", "theta_31", "theta_32", "theta_33",
         "CPU Time", "Converged", "# of Major Iter.", "# of Func. Eval.", 
         "# of Bellm. Iter.", "# of N-K Iter."]

results = pd.DataFrame(index=index, columns=columns)

In [ ]:
# Main loop to calculate the results for each run
for factor in discount_factor:
    # load simulated data 
    mat = scipy.io.loadmat("auxiliary_iskhakov_et_al_2016/RustBusTableXSimDataMC250_beta" + str(int(100000*factor)))
    
    for run in range(number_runs):
        data = process_data(mat, run, number_buses, number_periods)
        
        for start in range(starting_cost_params.shape[1]):
            # Adapt the Initiation Dictionairy of NFXP for this run
            init_dict_nfxp["model_specifications"]["discount_factor"] = factor
            init_dict_nfxp["optimizer"]["params"] = pd.DataFrame(starting_cost_params[:, start], columns=["value"])
            
            # Run NFXP using ruspy
            transition_result_nfxp, cost_result_nfxp = estimate(init_dict_nfxp, data)
            
            # store the results of this run
            results.loc[factor, run, start, "NFXP"] = process_result(
                "NFXP", transition_result_nfxp, cost_result_nfxp, number_states)
            
            # Adapt the Initiation Dictionairy of MPEC for this run
            init_dict_mpec["model_specifications"]["discount_factor"] = factor
            init_dict_mpec["optimizer"]["params"] = np.concatenate((
                starting_expected_value_fun, starting_cost_params[:, start]))
            
            # Run MPEC using ruspy
            transition_result_mpec, cost_result_mpec = estimate(init_dict_mpec, data)      
            
            # store the results of this run
            results.loc[
                factor, run, start, "MPEC"].loc[
                ~results.columns.isin(["# of Bellm. Iter.", "# of N-K Iter."])] = process_result(
                        "MPEC", transition_result_mpec, cost_result_mpec, number_states)

To give you an idea of what the above loop has just produced, let us have a look at the results.

In [5]:
results

RC theta_11   theta_30  theta_31  \
Discount Factor Run Start Approach                                          
0.9750          0   0     NFXP      8.89422  1.55117  0.0988235  0.442857   
                          MPEC      8.89422  1.55117  0.0988235  0.442857   
                    1     NFXP      8.89422  1.55117  0.0988235  0.442857   
                          MPEC      8.89422  1.55117  0.0988235  0.442857   
                    2     NFXP      8.89422  1.55117  0.0988235  0.442857   
...                                     ...      ...        ...       ...   
0.9999          249 2     MPEC      13.6492  2.89758  0.0947899   0.44958   
                    3     NFXP      13.6492  2.89758  0.0947899   0.44958   
                          MPEC      13.6492  2.89758  0.0947899   0.44958   
                    4     NFXP      13.6492  2.89758  0.0947899   0.44958   
                          MPEC      13.6491  2.89758  0.0947899   0.44958   

                                    theta_32   theta_33  CPU Time Converged  \
Discount Factor Run Start Approach                                            
0.9750          0   0     NFXP      0.446555  0.0112605  0.573507         1   
                          MPEC      0.446555  0.0112605     1.169         1   
                    1     NFXP      0.446555  0.0112605  0.657405         1   
                          MPEC      0.446555  0.0112605     0.749         1   
                    2     NFXP      0.446555  0.0112605  0.634032         1   
...                                      ...        ...       ...       ...   
0.9999          249 2     MPEC      0.441008  0.0146218     1.827         1   
                    3     NFXP      0.441008  0.0146218   2.97998         1   
                          MPEC      0.441008  0.0146218      1.53         1   
                    4     NFXP      0.441008  0.0146218   3.03964         1   
                          MPEC      0.441008  0.0146218     2.049         1   

                                   # of Major Iter. # of Func. Eval.  \
Discount Factor Run Start Approach                                     
0.9750          0   0     NFXP                    7                8   
                          MPEC                   23               29   
                    1     NFXP                    7                8   
                          MPEC                   15               17   
                    2     NFXP                    7                8   
...                                             ...              ...   
0.9999          249 2     MPEC                   28               35   
                    3     NFXP                   11               13   
                          MPEC                   23               35   
                    4     NFXP                   12               15   
                          MPEC                   32               45   

                                   # of Bellm. Iter. # of N-K Iter.  
Discount Factor Run Start Approach                                   
0.9750          0   0     NFXP                   180             57  
                          MPEC                   NaN            NaN  
                    1     NFXP                   180             57  
                          MPEC                   NaN            NaN  
                    2     NFXP                   180             57  
...                                              ...            ...  
0.9999          249 2     MPEC                   NaN            NaN  
                    3     NFXP                   280            280  
                          MPEC                   NaN            NaN  
                    4     NFXP                   320            320  
                          MPEC                   NaN            NaN  

[15000 rows x 12 columns]

---
### Table I from Su & Judd (2012)
---

Before we proceed to Table I in Iskhakov et al., we give out the means and standard deviations of our parameter estimations like it was done by Su and Judd (2012) in Table I.

In [18]:
# Create Table I from Su & Judd (2012) with the simulated values from Iskahkov et al. (2016)
columns_table_1 = ["RC", "theta_11", "theta_30", "theta_31", "theta_32", "theta_33"]
table_1_temp = results.loc[results["Converged"] == 1, columns_table_1].astype(float).groupby(
    level=["Discount Factor", "Approach"])

statistic = ["Mean", "Standard Deviation"]
index = pd.MultiIndex.from_product([discount_factor, approach, statistic],
                                   names=["Discount Factor", "Approach", "Statistic"])
table_1 = pd.DataFrame(index=index, columns=columns_table_1)
table_1.loc(axis=0)[:,:,"Mean"] = table_1_temp.mean()
table_1.loc(axis=0)[:,:,"Standard Deviation"] = table_1_temp.std()

In [19]:
table_1.astype(float).round(3)

RC  theta_11  theta_30  \
Discount Factor Approach Statistic                                        
0.9750          NFXP     Mean                11.908     2.507     0.094   
                         Standard Deviation   1.517     0.468     0.004   
                MPEC     Mean                11.908     2.507     0.094   
                         Standard Deviation   1.517     0.468     0.004   
0.9850          NFXP     Mean                11.986     2.534     0.094   
                         Standard Deviation   1.457     0.452     0.004   
                MPEC     Mean                11.986     2.534     0.094   
                         Standard Deviation   1.457     0.452     0.004   
0.9950          NFXP     Mean                11.891     2.508     0.094   
                         Standard Deviation   1.384     0.440     0.004   
                MPEC     Mean                11.891     2.508     0.094   
                         Standard Deviation   1.384     0.440     0.004   
0.9990          NFXP     Mean                11.874     2.513     0.094   
                         Standard Deviation   1.347     0.444     0.004   
                MPEC     Mean                11.874     2.513     0.094   
                         Standard Deviation   1.347     0.444     0.004   
0.9995          NFXP     Mean                11.847     2.508     0.094   
                         Standard Deviation   1.343     0.445     0.004   
                MPEC     Mean                11.849     2.509     0.094   
                         Standard Deviation   1.343     0.445     0.004   
0.9999          NFXP     Mean                11.815     2.499     0.094   
                         Standard Deviation   1.319     0.431     0.004   
                MPEC     Mean                11.815     2.498     0.094   
                         Standard Deviation   1.319     0.431     0.004   

                                             theta_31  theta_32  theta_33  
Discount Factor Approach Statistic                                         
0.9750          NFXP     Mean                   0.448     0.446     0.013  
                         Standard Deviation     0.006     0.007     0.001  
                MPEC     Mean                   0.448     0.446     0.013  
                         Standard Deviation     0.006     0.007     0.001  
0.9850          NFXP     Mean                   0.448     0.446     0.013  
                         Standard Deviation     0.006     0.007     0.001  
                MPEC     Mean                   0.448     0.446     0.013  
                         Standard Deviation     0.006     0.007     0.001  
0.9950          NFXP     Mean                   0.448     0.446     0.013  
                         Standard Deviation     0.006     0.007     0.001  
                MPEC     Mean                   0.448     0.446     0.013  
                         Standard Deviation     0.006     0.007     0.001  
0.9990          NFXP     Mean                   0.448     0.446     0.013  
                         Standard Deviation     0.006     0.007     0.001  
                MPEC     Mean                   0.448     0.446     0.013  
                         Standard Deviation     0.006     0.007     0.001  
0.9995          NFXP     Mean                   0.448     0.446     0.013  
                         Standard Deviation     0.006     0.007     0.001  
                MPEC     Mean                   0.448     0.446     0.013  
                         Standard Deviation     0.006     0.007     0.001  
0.9999          NFXP     Mean                   0.448     0.446     0.013  
                         Standard Deviation     0.006     0.007     0.001  
                MPEC     Mean                   0.448     0.446     0.013  
                         Standard Deviation     0.006     0.007     0.001

---
### Corresponding (reduced) results from Iskhakov et al. (2016)
---

To get an idea of the qualitiy of our estimates, we compare this now to the results obtained by Iskhakov et al. using their original matlab code for the implementation of their NFXP. Those results were not published in the paper.

In [12]:
# Create a Table with the results 
index = pd.MultiIndex.from_product([discount_factor, statistic],
                                   names=["Discount Factor", "Statistic"])
NFXP_Iskhakov = pd.DataFrame(index=index, columns=["RC", "theta_11"])
for factor in discount_factor:
    NFXP_Iskhakov_temp = scipy.io.loadmat(
        "auxiliary_iskhakov_et_al_2016/solution_iskhakov_beta_" + str(int(100000*factor)))[
        "result_jr87_" + str(int(100000*factor))]
    NFXP_Iskhakov.loc[factor, "Mean"] = NFXP_Iskhakov_temp.mean(axis=0)
    NFXP_Iskhakov.loc[factor, "Standard Deviation"] = NFXP_Iskhakov_temp.std(axis=0)

In [13]:
NFXP_Iskhakov.astype(float).round(3)

RC  theta_11
Discount Factor Statistic                           
0.9750          Mean                11.914     2.508
                Standard Deviation   1.517     0.468
0.9850          Mean                11.991     2.535
                Standard Deviation   1.457     0.452
0.9950          Mean                11.191     2.902
                Standard Deviation   1.188     0.473
0.9990          Mean                11.876     2.513
                Standard Deviation   1.346     0.444
0.9995          Mean                11.849     2.509
                Standard Deviation   1.342     0.445
0.9999          Mean                11.817     2.499
                Standard Deviation   1.319     0.431

---
### Table I from Iskhakov et al. (2016)
---

Lastly, below we create our results of Table I of Iskhakov et al. (2016). Please be aware that both our implementations are following the original setup closely but not perfectly. Especially notable is that both our implementation of MPEC as well as NFXP are somewhat inferior to those of the paper as we do not provided the analytical Hessian plus sparsity patterns for MPEC and in the case of the NFXP no back-and-forth-switching for the fixed point calculation.

In [14]:
# Create Table I from Iskhakov et al. (2016)
columns_table_2 = ["CPU Time", "Converged", "# of Major Iter.", "# of Func. Eval.", "# of Bellm. Iter.", "# of N-K Iter."]
table_2 = results[columns_table_2].astype(float).groupby(["Discount Factor", "Approach"]).mean()
table_2["Converged"] = (table_2["Converged"]*number_runs*starting_cost_params.shape[1]).astype(int)

In [24]:
table_2.astype(float).round(3)

CPU Time  Converged  # of Major Iter.  \
Discount Factor Approach                                          
0.9750          MPEC         1.151     1250.0            19.556   
                NFXP         1.286     1250.0            11.755   
0.9850          MPEC         1.187     1250.0            19.880   
                NFXP         1.306     1250.0            11.356   
0.9950          MPEC         1.352     1250.0            22.166   
                NFXP         1.185     1250.0            10.606   
0.9990          MPEC         1.613     1249.0            25.275   
                NFXP         1.556     1250.0            10.936   
0.9995          MPEC         1.754     1248.0            26.113   
                NFXP         2.626     1250.0            10.922   
0.9999          MPEC         1.861     1250.0            28.149   
                NFXP         2.778     1250.0            10.862   

                          # of Func. Eval.  # of Bellm. Iter.  # of N-K Iter.  
Discount Factor Approach                                                       
0.9750          MPEC                25.864                NaN             NaN  
                NFXP                14.122            301.733         104.399  
0.9850          MPEC                28.391                NaN             NaN  
                NFXP                13.581            291.187         105.620  
0.9950          MPEC                35.076                NaN             NaN  
                NFXP                12.666            272.932          97.727  
0.9990          MPEC                41.540                NaN             NaN  
                NFXP                12.925            278.025         138.394  
0.9995          MPEC                43.598                NaN             NaN  
                NFXP                12.903            277.585         250.746  
0.9999          MPEC                49.267                NaN             NaN  
                NFXP                12.858            276.620         270.001

### References

Gabler, J., "A Python Tool for the Estimation of (Structural) Econometric Models.", unpublished (2019), https://github.com/OpenSourceEconomics/estimagic

Iskhakov, F., Lee, J., Rust, J., Schjerning, B. and Seo, K. (2016), Comment on “Constrained Optimization Approaches to Estimation of Structural Models”. Econometrica, 84: 365-370. doi:10.3982/ECTA12605

Johnson, Steven G., The NLopt nonlinear-optimization package, http://github.com/stevengj/nlopt

Rust, John. "Optimal Replacement of GMC Bus Engines: An Empirical Model of Harold Zurcher." Econometrica 55, no. 5 (1987): 999-1033. Accessed June 7, 2020. doi:10.2307/1911259.

Rust, John. "Nested fixed point algorithm documentation manual." Unpublished Manuscript (6) (2000): 1-43.

Su, C.‐L. and Judd, K.L. (2012), Constrained Optimization Approaches to Estimation of Structural Models. Econometrica, 80: 2213-2230. doi:10.3982/ECTA7925

Wächter, A. and Biegler, L. T., On the Implementation of a Primal-Dual Interior Point Filter Line Search Algorithm for Large-Scale Nonlinear Programming, Mathematical Programming 106(1), pp. 25-57, 2006 (preprint), https://github.com/coin-or/Ipopt